In [ ]:
from skimage.io import imshow, imread
from skimage.color import rgb2gray
from skimage.filters import threshold_otsu, median
from skimage.measure import label, regionprops_table
from skimage.morphology import remove_small_objects, remove_small_holes, erosion
import numpy as np

## Read Image

In [ ]:
# read image
image = imread("../data/cleaned_dataset/arive_dantu/AV-S-001.jpg")
imshow(image);

## Convert RGB to Grayscale

In [ ]:
image_gray = rgb2gray(image)
imshow(image_gray, cmap="gray");

### Median Filtering

In [ ]:
image_median = median(image_gray)
imshow(image_median, cmap="gray")

### Thresholding (Binary Image)

In [ ]:
threshold = threshold_otsu(image_gray)
image_binary = image_gray > threshold
imshow(image_binary, cmap="gray");

### Erosion

In [ ]:
image_erosion = erosion(image_binary)
image_erosion = remove_small_objects(image_erosion, 50)
image_erosion = remove_small_holes(image_erosion, 50)
imshow(image_erosion, cmap="gray")

### Inverse Image

In [ ]:
inverted_img = np.invert(image_erosion)
imshow(inverted_img, cmap="gray")

### Feature Extraction

In [ ]:
properties = [
    "area",
    "eccentricity",
    "major_axis_length",
    "minor_axis_length",
    "perimeter",
]

labels = label(inverted_img)
props = regionprops_table(labels, inverted_img, properties=properties)
props

## Test Predict

In [ ]:
import numpy as np
import pickle

with open("../temp/scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

temp = {"saya": 0.1, "coba": 2.0, "lagi": 3.0, "ya": 4.0, "saka": 5.0}
temp = np.array(list(temp.values()))
scaler.transform(temp.reshape(1, -1))

# ====================

with open("../temp/knn_model.pkl", "rb") as f:
    model = pickle.load(f)

predict = model.predict_proba(temp.reshape(1, -1))
predict

# ====================

import json

with open("../temp/class_mapping.json", "r") as f:
    class_mapping = json.load(f)


top_5 = np.argsort(predict[0])[-5:][::-1]
top_5_dict = {class_mapping[str(i)]: round(predict[0][i], 2) for i in top_5}
top_5_dict